# 🔥 FILE 1-D: GPU, Debugging & Exercises

**PHẦN 1 - FOUNDATION (BEGINNER) - FINAL**

---

## 📋 Nội Dung

✅ CPU vs GPU - Khi nào dùng gì?

✅ Chuyển model và data lên GPU

✅ Device management best practices

✅ Common errors của người mới

✅ Debugging techniques

✅ Performance tips

✅ Tổng hợp bài tập Foundation

---

## 🎯 Mục Tiêu

Sau file này, bạn sẽ:
- Biết cách sử dụng GPU hiệu quả
- Tự debug được các lỗi thường gặp
- Hoàn thành Foundation level
- Sẵn sàng cho Intermediate level

---

## ⏱️ Thời Gian Học: 2-3 giờ

---

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time

print(f"PyTorch version: {torch.__version__}")
print("✅ Ready to learn GPU & Debugging!")

---

# 1️⃣ CPU vs GPU

## Sự Khác Biệt

### CPU (Central Processing Unit)
- 🔹 Ít cores (~4-16)
- 🔹 Clock speed cao
- 🔹 Tốt cho tác vụ tuần tự
- 🔹 Linh hoạt, đa năng

### GPU (Graphics Processing Unit)
- 🔸 Hàng nghìn cores
- 🔸 Clock speed thấp hơn
- 🔸 Tốt cho tác vụ song song
- 🔸 Chuyên biệt cho ma trận

## Deep Learning & GPU

Neural networks = Rất nhiều phép toán ma trận song song

→ **GPU nhanh hơn 10-100 lần!**

### Khi Nào Dùng GPU?

✅ **Nên dùng GPU:**
- Model lớn (>1M parameters)
- Dữ liệu nhiều (>10k samples)
- Training lâu
- CNN, Transformer, RNN

❌ **Không cần GPU:**
- Model nhỏ (<100k parameters)
- Dữ liệu ít
- Prototyping nhanh
- Linear models đơn giản

---

# 2️⃣ Kiểm Tra GPU

In [ ]:
print("=" * 70)
print("KIỂM TRA GPU")
print("=" * 70)

# Kiểm tra CUDA
cuda_available = torch.cuda.is_available()
print(f"\nCUDA available: {cuda_available}")

if cuda_available:
    print(f"\nCUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU index: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    
    # Memory info
    total_memory = torch.cuda.get_device_properties(0).total_memory
    print(f"\nGPU memory:")
    print(f"  Total: {total_memory / 1e9:.2f} GB")
    print(f"  Allocated: {torch.cuda.memory_allocated() / 1e9:.4f} GB")
    print(f"  Cached: {torch.cuda.memory_reserved() / 1e9:.4f} GB")
else:
    print("\nNo GPU detected")
    print("→ Using CPU (hoàn toàn OK để học!)")
    print("→ GPU chỉ cần khi train model lớn")

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ Using device: {device}")

---

# 3️⃣ Chuyển Tensor Lên GPU

In [ ]:
print("=" * 70)
print("CHUYỂN TENSOR LÊN GPU")
print("=" * 70)

# Tạo tensor trên CPU
x_cpu = torch.randn(3, 3)
print(f"\n1. Tensor trên CPU:")
print(f"   Device: {x_cpu.device}")
print(f"   Data:\n{x_cpu}")

if torch.cuda.is_available():
    print("\n" + "-" * 70)
    print("CÁCH 1: .to(device)")
    print("-" * 70)
    x_gpu = x_cpu.to(device)
    print(f"Device: {x_gpu.device}")
    
    print("\n" + "-" * 70)
    print("CÁCH 2: .cuda()")
    print("-" * 70)
    x_gpu2 = x_cpu.cuda()
    print(f"Device: {x_gpu2.device}")
    
    print("\n" + "-" * 70)
    print("CÁCH 3: Tạo trực tiếp trên GPU")
    print("-" * 70)
    x_gpu3 = torch.randn(3, 3, device=device)
    print(f"Device: {x_gpu3.device}")
    
    print("\n" + "-" * 70)
    print("CHUYỂN VỀ CPU")
    print("-" * 70)
    x_back = x_gpu.cpu()
    print(f"Device: {x_back.device}")
    
    print("\n💡 Khuyến nghị: Dùng .to(device) để linh hoạt")
else:
    print("\nGPU not available, skipping GPU examples")

---

# 4️⃣ Chuyển Model Lên GPU

In [ ]:
print("=" * 70)
print("CHUYỂN MODEL LÊN GPU")
print("=" * 70)

# Tạo model
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleModel()
print(f"\nModel ban đầu (CPU):")
for name, param in model.named_parameters():
    print(f"  {name}: {param.device}")

# Chuyển lên GPU
model = model.to(device)
print(f"\nSau khi .to(device):")
for name, param in model.named_parameters():
    print(f"  {name}: {param.device}")

print("\n✅ TẤT CẢ parameters đã chuyển lên GPU!")

---

# 5️⃣ Training Loop Với GPU

In [ ]:
print("=" * 70)
print("TRAINING LOOP VỚI GPU")
print("=" * 70)

# Tạo data
X = torch.randn(100, 10)
y = torch.randn(100, 1)

# Tạo model
model = SimpleModel().to(device)  # ← Chuyển model lên GPU
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(f"\nDevice: {device}")
print(f"Model device: {next(model.parameters()).device}")

# Training loop
n_epochs = 50

for epoch in range(n_epochs):
    # ⚠️ QUAN TRỌNG: Chuyển data lên cùng device với model
    X_device = X.to(device)
    y_device = y.to(device)
    
    # Forward
    y_pred = model(X_device)
    loss = criterion(y_pred, y_device)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}")

print("\n✅ Training hoàn thành!")

print("\n" + "=" * 70)
print("💡 QUAN TRỌNG")
print("=" * 70)
print("""
Model và Data phải ở CÙNG device!

Cách tốt nhất:
1. Định nghĩa device một lần ở đầu
2. Chuyển model: model.to(device)
3. Chuyển data trong loop: X.to(device), y.to(device)
""")

---

# 6️⃣ So Sánh Tốc Độ CPU vs GPU

In [ ]:
print("=" * 70)
print("BENCHMARK: CPU vs GPU")
print("=" * 70)

# Tạo ma trận lớn
size = 5000
a_cpu = torch.randn(size, size)
b_cpu = torch.randn(size, size)

# Benchmark CPU
print(f"\nMatrix size: {size} × {size}")
print("\nTesting CPU...")
start = time.time()
c_cpu = torch.matmul(a_cpu, b_cpu)
cpu_time = time.time() - start
print(f"CPU time: {cpu_time:.4f} seconds")

# Benchmark GPU (nếu có)
if torch.cuda.is_available():
    a_gpu = a_cpu.to(device)
    b_gpu = b_cpu.to(device)
    
    # Warm-up GPU
    _ = torch.matmul(a_gpu, b_gpu)
    torch.cuda.synchronize()
    
    print("\nTesting GPU...")
    start = time.time()
    c_gpu = torch.matmul(a_gpu, b_gpu)
    torch.cuda.synchronize()  # Đợi GPU hoàn thành
    gpu_time = time.time() - start
    print(f"GPU time: {gpu_time:.4f} seconds")
    
    print(f"\n🚀 Speedup: {cpu_time / gpu_time:.2f}x")
    
    # Visualize
    plt.figure(figsize=(8, 5))
    plt.bar(['CPU', 'GPU'], [cpu_time, gpu_time], color=['blue', 'green'])
    plt.ylabel('Time (seconds)')
    plt.title('CPU vs GPU Performance')
    plt.grid(True, alpha=0.3, axis='y')
    for i, v in enumerate([cpu_time, gpu_time]):
        plt.text(i, v + 0.01, f'{v:.3f}s', ha='center', fontweight='bold')
    plt.show()
else:
    print("\nGPU not available for comparison")

print("\n💡 Lưu ý:")
print("  - GPU nhanh hơn nhiều với ma trận lớn")
print("  - Với ma trận nhỏ, CPU có thể nhanh hơn (overhead)")

---

# 7️⃣ Common Errors - Lỗi Thường Gặp

## 7.1 Device Mismatch

In [ ]:
print("=" * 70)
print("LỖI 1: DEVICE MISMATCH")
print("=" * 70)

if torch.cuda.is_available():
    model = nn.Linear(10, 5).to(device)
    x = torch.randn(3, 10)  # ← Trên CPU
    
    print(f"\nModel device: {next(model.parameters()).device}")
    print(f"Input device: {x.device}")
    
    try:
        y = model(x)  # ❌ LỖI!
    except RuntimeError as e:
        print(f"\n❌ ERROR: {str(e)[:80]}...")
    
    print("\n" + "=" * 70)
    print("✅ GIẢI PHÁP")
    print("=" * 70)
    x = x.to(device)  # Chuyển input lên GPU
    y = model(x)  # OK!
    print(f"\nInput device: {x.device}")
    print(f"Output device: {y.device}")
    print("✅ Success!")
else:
    print("\nGPU not available, skipping this example")

## 7.2 Gradient Not Zeroed

In [ ]:
print("=" * 70)
print("LỖI 2: QUÊN ZERO GRADIENT")
print("=" * 70)

model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

x = torch.tensor([[1.0]])
y = torch.tensor([[2.0]])

print("\nKhông zero gradient:")
for i in range(3):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    # ❌ QUÊN optimizer.zero_grad()
    optimizer.step()
    print(f"Iteration {i+1}: grad = {model.weight.grad.item():.4f}")

print("\n⚠️ Gradient cộng dồn → Kết quả SAI!")

print("\n" + "=" * 70)
print("✅ ĐÚNG CÁCH")
print("=" * 70)

model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for i in range(3):
    optimizer.zero_grad()  # ← PHẢI CÓ!
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    print(f"Iteration {i+1}: grad = {model.weight.grad.item():.4f}")

print("\n✅ Gradient đúng mỗi iteration")

## 7.3 Shape Mismatch

In [ ]:
print("=" * 70)
print("LỖI 3: SHAPE MISMATCH")
print("=" * 70)

model = nn.Linear(10, 5)
x = torch.randn(3, 5)  # ❌ Wrong: should be (3, 10)

print(f"\nModel expects: (batch, 10)")
print(f"Input shape: {x.shape}")

try:
    y = model(x)
except RuntimeError as e:
    print(f"\n❌ ERROR: {str(e)[:80]}...")

print("\n" + "=" * 70)
print("✅ GIẢI PHÁP")
print("=" * 70)
x = torch.randn(3, 10)  # Đúng shape
y = model(x)
print(f"\nInput: {x.shape}")
print(f"Output: {y.shape}")
print("✅ Success!")

## 7.4 In-place Operations

In [ ]:
print("=" * 70)
print("LỖI 4: IN-PLACE OPERATIONS")
print("=" * 70)

x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x ** 2

try:
    x[0] = 999  # ❌ In-place operation
    y.backward(torch.ones_like(y))
except RuntimeError as e:
    print(f"❌ ERROR: {str(e)[:100]}...")

print("\n" + "=" * 70)
print("✅ GIẢI PHÁP")
print("=" * 70)
print("""
Tránh in-place operations (+=, *=, []=) trên tensors có requires_grad=True

❌ Bad:  x[0] = 999
✅ Good: x = torch.cat([torch.tensor([999.0]), x[1:]])

❌ Bad:  x += 1
✅ Good: x = x + 1
""")

---

# 8️⃣ Debugging Techniques

## 8.1 Print Shapes

In [ ]:
print("=" * 70)
print("DEBUGGING: PRINT SHAPES")
print("=" * 70)

class DebugModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        print(f"  Input: {x.shape}")
        
        x = self.fc1(x)
        print(f"  After fc1: {x.shape}")
        
        x = torch.relu(x)
        print(f"  After ReLU: {x.shape}")
        
        x = self.fc2(x)
        print(f"  After fc2: {x.shape}")
        
        x = torch.relu(x)
        x = self.fc3(x)
        print(f"  Output: {x.shape}")
        
        return x

model = DebugModel()
x = torch.randn(5, 10)

print("\nForward pass:")
y = model(x)

print("\n💡 Print shapes giúp debug shape mismatch errors!")

## 8.2 Check Gradients

In [ ]:
print("=" * 70)
print("DEBUGGING: CHECK GRADIENTS")
print("=" * 70)

model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 1)
)

x = torch.randn(3, 10)
y = torch.randn(3, 1)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Forward + backward
y_pred = model(x)
loss = criterion(y_pred, y)
loss.backward()

# Check gradients
print("\nGradient check:")
for name, param in model.named_parameters():
    if param.grad is not None:
        grad_norm = param.grad.norm().item()
        print(f"{name:20s}: grad_norm = {grad_norm:.6f}")
    else:
        print(f"{name:20s}: ⚠️ No gradient!")

print("\n💡 Nếu gradient = 0 hoặc None → có vấn đề!")
print("   Có thể do: Dead ReLU, vanishing gradient, etc.")

## 8.3 Gradient Clipping

In [ ]:
print("=" * 70)
print("GRADIENT CLIPPING")
print("=" * 70)

print("""
Gradient clipping ngăn exploding gradients
Hữu ích cho RNN, LSTM, GRU
""")

model = nn.Linear(10, 5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

x = torch.randn(3, 10)
y = torch.randn(3, 5)

# Training với gradient clipping
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y)
loss.backward()

# Clip gradients
max_norm = 1.0
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

optimizer.step()

print(f"\n✅ Gradients clipped to max_norm={max_norm}")
print("\nCách dùng:")
print("""
loss.backward()
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
optimizer.step()
""")

---

# 9️⃣ Best Practices

## Checklist

In [ ]:
print("=" * 70)
print("✅ PYTORCH BEST PRACTICES")
print("=" * 70)

print("""
1. DEVICE MANAGEMENT
   ✅ Định nghĩa device ở đầu code
   ✅ Chuyển model một lần: model.to(device)
   ✅ Chuyển data trong loop: x.to(device)
   ❌ Không mix CPU và GPU tensors

2. GRADIENT MANAGEMENT
   ✅ Luôn zero_grad() trước backward()
   ✅ Dùng optimizer.zero_grad() thay vì manual
   ❌ Không quên zero gradients

3. TRAINING LOOP
   ✅ Thứ tự: forward → loss → zero_grad → backward → step
   ✅ Dùng torch.no_grad() khi eval
   ✅ Set model.train() / model.eval()

4. MEMORY
   ✅ Dùng torch.no_grad() khi không cần gradient
   ✅ Clear cache: torch.cuda.empty_cache()
   ✅ Del tensors không dùng nữa
   ❌ Không lưu toàn bộ history

5. REPRODUCIBILITY
   ✅ Set seed: torch.manual_seed(42)
   ✅ Set CUDA seed: torch.cuda.manual_seed(42)
   ✅ Deterministic operations (nếu cần)

6. CODE ORGANIZATION
   ✅ Tách model definition ra class riêng
   ✅ Tách training loop ra function
   ✅ Tách data loading ra function
   ✅ Comment code rõ ràng

7. DEBUGGING
   ✅ Print shapes thường xuyên
   ✅ Check gradients
   ✅ Visualize loss curves
   ✅ Start simple, add complexity gradually
""")

---

# 🔟 Template Code - Sử Dụng Ngay

In [ ]:
print("=" * 70)
print("TEMPLATE: COMPLETE TRAINING PIPELINE")
print("=" * 70)

# ============= SETUP =============
import torch
import torch.nn as nn
import torch.optim as optim

# Set seed for reproducibility
torch.manual_seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")

# ============= MODEL =============
class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# ============= DATA =============
X = torch.randn(100, 10)
y = torch.randn(100, 1)

# ============= TRAINING SETUP =============
model = MyModel(10, 64, 1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ============= TRAINING LOOP =============
n_epochs = 50
for epoch in range(n_epochs):
    # Move data to device
    X_device = X.to(device)
    y_device = y.to(device)
    
    # Forward
    y_pred = model(X_device)
    loss = criterion(y_pred, y_device)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Log
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}")

print("\n✅ Template completed!")
print("\n💡 Copy template này làm starting point cho projects!")

---

# 1️⃣1️⃣ Bài Tập Tổng Hợp Foundation

## Bài 1: Linear Regression Hoàn Chỉnh

**Đề bài:**
1. Tạo dữ liệu: y = 2x₁ + 3x₂ - 4x₃ + 5 + noise
2. Chia train/test 80/20
3. Build model bằng nn.Linear
4. Train 100 epochs
5. Evaluate trên test set
6. Visualize: loss curves, predictions vs true
7. So sánh learned parameters vs true

**Yêu cầu:**
- Dùng torch.nn
- Dùng Adam optimizer
- MSE loss
- Chạy trên GPU (nếu có)

## Bài 2: Binary Classification

**Đề bài:**
1. Tạo dữ liệu binary classification (dùng make_circles)
2. Build MLP: Input → 32 → 16 → 2 (classes)
3. Dùng ReLU activation
4. CrossEntropyLoss
5. Train và track accuracy
6. Visualize decision boundary
7. Confusion matrix

**Yêu cầu:**
- Custom nn.Module
- Training accuracy > 90%
- Plot decision boundary đẹp

## Bài 3: Multi-class Classification

**Đề bài:**
1. Tạo 4 classes data (dùng make_blobs)
2. Build deeper MLP: 2 → 64 → 32 → 16 → 4
3. Thử các activation: ReLU, Tanh, LeakyReLU
4. So sánh performance
5. Plot loss và accuracy cho mỗi activation

**Yêu cầu:**
- 3 models với activations khác nhau
- Training curves để so sánh
- Kết luận activation nào tốt nhất

## Bài 4: Overfitting vs Regularization

**Đề bài:**
1. Tạo data nhỏ (100 samples, 20 features)
2. Build model rất lớn (20 → 256 → 128 → 64 → 1)
3. Train đến khi overfit (train loss << val loss)
4. Thêm L2 regularization (weight_decay)
5. So sánh với/không regularization

**Yêu cầu:**
- Plot train/val loss curves
- Chứng minh overfitting
- Chứng minh regularization giúp giảm overfit

## Bài 5: Learning Rate Experiment

**Đề bài:**
1. Tạo data regression đơn giản
2. Train với learning rates: [0.0001, 0.001, 0.01, 0.1, 1.0]
3. So sánh:
   - Tốc độ hội tụ
   - Final loss
   - Stability
4. Tìm LR tối ưu
5. Plot tất cả loss curves trên cùng graph

**Yêu cầu:**
- 5 experiments
- Comprehensive comparison
- Clear conclusion

---

# ✅ Tổng Kết FILE 1-D & Foundation Level

## FILE 1-D: Bạn Đã Học Được

✅ **CPU vs GPU** - Khi nào dùng gì

✅ **Device management** - Chuyển model và data

✅ **GPU benchmark** - So sánh tốc độ

✅ **Common errors** - Device mismatch, gradient issues, shapes

✅ **Debugging** - Print shapes, check gradients, clip gradients

✅ **Best practices** - Checklist đầy đủ

✅ **Template code** - Sử dụng ngay

---

## 🎉 HOÀN THÀNH FOUNDATION LEVEL!

### Bạn Đã Học Được (FILE 1-A đến 1-D):

📘 **FILE 1-A**: Tensor basics
- Tensor operations
- Indexing, slicing, reshaping
- Broadcasting
- Matrix multiplication

📘 **FILE 1-B**: Autograd & Training
- Gradient computation
- backward(), zero_grad()
- Gradient descent
- Linear regression từ đầu

📘 **FILE 1-C**: torch.nn
- nn.Module
- Layers và activations
- Loss functions
- Optimizers
- MLP đầu tiên

📘 **FILE 1-D**: GPU & Production
- GPU usage
- Debugging
- Best practices
- Production-ready code

---

## Bạn Giờ Có Thể:

✅ Hiểu rõ cách PyTorch hoạt động

✅ Xây dựng neural networks

✅ Train models hiệu quả

✅ Debug khi gặp lỗi

✅ Sử dụng GPU

✅ Viết code production-ready

---

## Tiếp Theo: INTERMEDIATE LEVEL

📗 **FILE 2-A: Dataset, DataLoader & Training Loop**
- Custom Dataset
- DataLoader
- Training/Validation split
- Full training pipeline

📗 **FILE 2-B: Optimizer, Activation & Regularization**
- Advanced optimizers
- More activations
- Dropout, BatchNorm
- Regularization techniques

📗 **FILE 2-C: CNN & Model Checkpoint**
- Convolutional layers
- Image classification
- Save/load models
- Transfer learning basics

---

## 🏆 Chúc Mừng!

Bạn đã hoàn thành **Foundation Level**!

Bạn giờ có nền tảng vững chắc để:
- Học các kỹ thuật nâng cao
- Build projects thực tế
- Tham gia competitions
- Đọc hiểu research papers

**Keep learning and building! 🚀**

---